In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import pandas as pd

In [ ]:
os.environ['WDM_LOG_LEVEL'] = '0'

# 
s = Service(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument('--disable-gpu')

driver = webdriver.Chrome(service=s, chrome_options=chrome_options)

driver.set_page_load_timeout(100)

url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/deng.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/chikun.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/zika.def"
driver.get(url)

WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]')))

# Linha
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[1]/select/option[1]').click()
# Coluna
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[2]/select/option[4]').click()
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[2]/select/option[4]').click()
# Periodo
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[2]').click()
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[1]').click()

# Mostra
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()


WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/table/tbody/tr/td[1]/a')))

html = driver.page_source

driver.quit()

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

tabdados = soup.select(".tabdados tbody tr td ")

tabdados = list(map(lambda node: node.get_text().strip(), tabdados))

col_tabdados = soup.select(".tabdados th ")
col_tabdados = list(map(lambda node: node.get_text().strip(), col_tabdados))

In [ ]:
tam_lin = int(len(tabdados)/len(col_tabdados))
tam_col = len(col_tabdados)

aux = []
for i in range (tam_lin):
    aux_2 = []
    for j in range (tam_col):  
        aux_2.append(tabdados.pop(0))
    aux.append(aux_2)

df = pd.DataFrame(aux, columns=col_tabdados)

In [ ]:
df = df.drop(columns=['Total'])
df = df.drop(0)
df = df.reset_index()
df = df.drop(columns=['index'])

In [ ]:
tam = len(df)
for i in range (tam):
    aux = str(df.iloc[:, 1][i])
    aux = aux.split(".")
    if len(aux) > 1:
        while len(aux) > 1:
            aux[0] += aux.pop(1)
        df.iloc[:, 1][i] = aux[0]

df.iloc[:, 1] = df.iloc[:, 1].astype('int')

In [ ]:
aux = list(df.iloc[:, 0])

In [ ]:
for i in range (len(aux)):
    aux[i] = aux[i].split(" ")
    while (len(aux[i]) > 2):
        aux[i][1] = aux[i][1] + " " + aux[i][2]
        aux[i].pop(2)


In [ ]:
aux = pd.DataFrame(aux, columns=["Codigo", "Município"])

In [ ]:
df["Codigo"] = aux["Codigo"]
df["Município"] = aux["Município"]

order = ["Codigo", "Município", df.columns[1]]

df = df[order]

In [ ]:
df.to_csv("../scr/casos_municipio.csv", encoding='utf-8')